In [1]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='llama3.2:1b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

The sky appears blue to us because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century. Here's a simplified explanation:

When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light, which means they scatter the light in all directions.

The shorter wavelengths of light, like blue and violet, are scattered more than the longer wavelengths, like red and orange. This is because the smaller molecules have a greater tendency to absorb and scatter the shorter wavelengths.

As a result, the blue light is dispersed throughout the atmosphere, giving the sky its blue color. The exact shade of blue can vary depending on atmospheric conditions, such as pollution, dust, and water vapor, but overall, the scattering effect dominates.

It's worth noting that during sunrise and sunset, the sky of

In [3]:
import whisper

# Load the model
model = whisper.load_model("tiny")  # Options: tiny, base, small, medium, large

# Transcribe an audio file
result = model.transcribe("./audio/speech.mp3")  # Replace with your audio file path

# Print the result
print("Transcription:", result['text'])

/opt/anaconda3/lib/python3.9/site-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:  How are you?


In [4]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

# Run TTS
# ❗ Since this model is multi-lingual voice cloning model, we must set the target speaker_wav and language
# Text to speech list of amplitude values as output
wav = tts.tts(text="Hello world!", speaker_wav="audio.wav", language="en")
# # Text to speech to a file
# tts.tts_to_file(text="Hello world!", speaker_wav="my/cloning/audio.wav", language="en", file_path="output.wav")

ModuleNotFoundError: No module named 'TTS'

In [5]:
import ChatTTS
import torch
import torchaudio

chat = ChatTTS.Chat()
chat.load(compile=False) # Set to True for better performance

texts = ["Hope you are doing great in this fine morning."]

wavs = chat.infer(texts)

for i in range(len(wavs)):
    """
    In some versions of torchaudio, the first line works but in other versions, so does the second line.
    """
    try:
        torchaudio.save(f"basic_output{i}.wav", torch.from_numpy(wavs[i]).unsqueeze(0), 24000)
    except:
        torchaudio.save(f"basic_output{i}.wav", torch.from_numpy(wavs[i]), 24000)

text:   4%|▍         | 16/384(max) [00:00, 40.36it/s]
code:   6%|▌         | 126/2048(max) [00:01, 76.43it/s]


In [ ]:
import torch
from TTS.api import TTS
import sounddevice as sd
import soundfile as sf
import numpy as np

class CoquiTTSWrapper:
    def __init__(self, model_name="tts_models/en/ljspeech/vits", device="cpu"):
        """Initialize TTS with specified model"""
        try:
            self.device = "cuda" if torch.cuda.is_available() and device == "cuda" else "cpu"
            self.tts = TTS(model_name=model_name, progress_bar=True).to(self.device)
        except Exception as e:
            raise Exception(f"Failed to initialize TTS: {str(e)}")
        
    def list_models(self):
        """List available models"""
        return TTS().list_models()
    
    def synthesize(self, text, output_path=None, sample_rate=22050):
        """Generate speech from text"""
        try:
            wav = self.tts.tts(text=text)
            
            if output_path:
                sf.write(output_path, wav, sample_rate)
            
            return wav, sample_rate
            
        except Exception as e:
            raise Exception(f"Speech synthesis failed: {str(e)}")
    
    def play_audio(self, wav, sample_rate=22050):
        """Play audio using sounddevice"""
        try:
            sd.play(wav, sample_rate)
            sd.wait()
        except Exception as e:
            raise Exception(f"Audio playback failed: {str(e)}")

def main():
    # Install required packages if not already installed:
    # pip install TTS torch sounddevice soundfile numpy
    
    try:
        # Initialize TTS
        tts = CoquiTTSWrapper()
        
        # Generate and play speech
        text = "Hope you're doing great in this fine morning!"
        wav, sr = tts.synthesize(text, "output.wav")
        tts.play_audio(wav, sr)
        
        # List available models
        print("\nAvailable models:")
        for model in tts.list_models():
            if "en" in model:  # Filter for English models
                print(f"- {model}")
                
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
import sounddevice as sd
import time

def generate_tone(frequency, duration, amplitude=0.2, sample_rate=44100):
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    return amplitude * np.sin(2 * np.pi * frequency * t)

# Generate two tones for "do-dom"
tone1 = generate_tone(440, 0.3)  # First tone: 440 Hz (A4)
tone2 = generate_tone(500, 0.3)  # Second tone: 392 Hz (G4)

# Concatenate with a short silence in between
silence = np.zeros(int(0.1 * 44100))  # 50 ms silence
siri_sound = np.concatenate((tone1, silence, tone2))

# Play the sound
sd.play(siri_sound, samplerate=44100)
sd.wait()
